## Commodity Futures RIC Search

### Overview

This notebook, demosntrates how to reconstruct comodity futures RICs and get histoical prices for futures contracts, including those already expired using [Refinitiv Data Libraries for Python](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python). Additionally, we will show how to request chain contracts, use continuation RICs and request historical prices on its constituents. Although LSEG deos provide the continuation RICs through which the market particpants can get the historical prices, LSEG rolls the contracts on the last trading day. Whereas the market paricipants may want to apply their own rolling logic. As the prices from individual futures are much closer to reality than simply using a generically rolled series, we attempt to build a Python class which returns the futures RIC providing the root code, expiration month and year as an input.

#### Learn more

To learn more about the Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.lseg.com/iam/register) and [logging](https://developers.lseg.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/learning), 
 [Documentation](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/docs)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
this [Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The LSEG Developer Community will be happy to help. 

----

In [1]:
import refinitiv.data as rd
import calendar
import pandas as pd
from refinitiv.data.discovery import Chain
from datetime import datetime

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x12dc84e80 {name='workspace'}>

### Using Chain function to get individual rics belonging the futures chain

Requesting individual contacts for LCO. Please note that these are going to be the contracts which are active at the time of the request.

In [3]:
lco = Chain(name="0#LCO:")
print(lco.constituents)

['LCOTOT', 'LCOV4', 'LCOX4', 'LCOZ4', 'LCOF5', 'LCOG5', 'LCOH5', 'LCOJ5', 'LCOK5', 'LCOM5', 'LCON5', 'LCOQ5', 'LCOU5', 'LCOV5', 'LCOX5', 'LCOZ5', 'LCOF6', 'LCOG6', 'LCOH6', 'LCOJ6', 'LCOK6', 'LCOM6', 'LCON6', 'LCOQ6', 'LCOU6', 'LCOV6', 'LCOX6', 'LCOZ6', 'LCOF7', 'LCOG7', 'LCOH7', 'LCOJ7', 'LCOK7', 'LCOM7', 'LCON7', 'LCOQ7', 'LCOU7', 'LCOV7', 'LCOX7', 'LCOZ7', 'LCOF8', 'LCOG8', 'LCOH8', 'LCOJ8', 'LCOK8', 'LCOM8', 'LCON8', 'LCOQ8', 'LCOU8', 'LCOV8', 'LCOX8', 'LCOZ8', 'LCOF9', 'LCOG9', 'LCOH9', 'LCOJ9', 'LCOK9', 'LCOM9', 'LCON9', 'LCOQ9', 'LCOU9', 'LCOV9', 'LCOX9', 'LCOZ9', 'LCOF0', 'LCOG0', 'LCOH0', 'LCOJ0', 'LCOK0', 'LCOM0', 'LCON0', 'LCOQ0', 'LCOU0', 'LCOV0', 'LCOX0', 'LCOZ0', 'LCOF31', 'LCOG31', 'LCOH31']


We can now request historical prices for one of these instruments. Let's say we want historical prices for LCOZ4 (expiring on December 2024) going back to 2010.

In [4]:
lco_prices = rd.get_history(universe = 'LCOZ4', fields = ['SETTLE','TRDPRC_1', 'BID', 'ASK'],  start = '2010-01-01', count = 10000)
lco_prices

LCOZ4,SETTLE,TRDPRC_1,BID,ASK
Date,,,,
2017-11-01,56.6,<NA>,<NA>,<NA>
2017-11-02,56.61,<NA>,<NA>,<NA>
2017-11-03,56.72,<NA>,<NA>,<NA>
2017-11-06,57.14,<NA>,<NA>,<NA>
2017-11-07,57.28,<NA>,<NA>,<NA>
...,...,...,...,...
2024-08-14,78.51,78.63,78.69,78.76
2024-08-15,79.6,79.54,79.49,79.54
2024-08-16,78.29,78.24,78.2,78.28


What we can see is that the data starts from November 2017 despite we have requested prices from 2010 onwards. That is because the particular contract is actually created on that day and in order to get the prices prior to that we need either:

    a. use futures continuation RICs
    b. get/reconstruct the expired futures contract RIC

### Using continuation RICs

According to Refinitiv RIC rules (can be accessed in LSEG Workspace by typing "RIC RULES" in the search box then selecting "Futures and Options" then "Fuure Continuation RICs") Continuation RICs for Futures automatically roll over on or before expiry. These can be for the front month as well as 2nd, 3rd, 4th, etc.  There are distinct codes for pricing, volume, and open interest data:

    c - General continuation for each record in the futures chain
    cm - Quarterly months continuation with roll over on the expiry date
    cm1t - Quarterly months continuation with roll over on the last day of the month before contract expiration
    v - Volume continuation
    cv - Continuation on future records based on Volume
    oi - Open Interest continuation
    1 - Contract month; 1 is the front month, 2 the following, etc This works the same for quarterly contracts as well.

Below let's request prices for general continuation for LCO.

In [5]:
lco_prices = rd.get_history(universe = 'LCOc1', fields = ['SETTLE','TRDPRC_1', 'BID', 'ASK'], start = '2010-10-01', count = 10000)
lco_prices

LCOc1,SETTLE,TRDPRC_1,BID,ASK
Date,,,,
2010-10-04,83.28,83.3,<NA>,<NA>
2010-10-05,84.84,84.65,<NA>,<NA>
2010-10-06,85.06,85.3,<NA>,<NA>
2010-10-07,83.43,83.11,<NA>,<NA>
2010-10-08,84.03,84.19,<NA>,<NA>
...,...,...,...,...
2024-08-14,79.76,80.03,79.99,80.04
2024-08-15,81.04,80.97,80.93,80.98
2024-08-16,79.68,79.58,79.58,79.64


As we can see the pricing data goes back to the point we asked for in our request, however, the continuation RICs are rolled on the last trading day, whereas many market participants may decide to roll with a different assumption to get closest possible prices. This can be achieved through individual futures contracts which are already expired only.

### Get/reconstruct the expired futures contract RICs

To reconstruct RICs for expired futures we need to follow RIC construction rules (can be accessed in LSEG Workspace by typing "RIC RULES" in the search box then selecting "Futures and Options" then "Futures RICs and Delivery Codes" and "Expired Futures Historical Data RICs"). Below is the construction logic for active and expired comodity futures contracts:

Active:

    S - Root Code
    Z - Delivery Month Code
    5 - Expiry Year Code

Expired

    S - RIC Root Code for Soybeans
    U - Month Code
    4 - Last digit of year
    ^ - Carat symbol
    1 - Decade of the expiry of the contract
    SU4^1 - Completed code (Searchable in the Command Line as well)

It should also be noted that there have been a RIC structure change (introduced two-digit year code) related to the year code introduced in 2024 for set of RICs. RIC structure data notification related to change can be found [here](https://myaccount.lseg.com/en/searchpage?_charset_=UTF-8&searchLanguage=en-us&q=RIC+Structure+Change+for+Futures+and+Spreads+Contracts&sp_cs=UTF-8&sp_&myproduct=false&page=1).

Good thing is we can access the expired futures RICs via the Search function of the Data Libraries API and only thing we need to do is to build the proper filtering following the RIC construction RULES above.

Below is a custom Python class which uses Search capabilities of the Data Library to get the active and expired future contracts RICs. The Class constructs the components of future RICs, form a search query and request data using the Search. The class is also using a constant named MONTH_CODES which lists month codes and month numbers as key/value pairs of a dictionary. Nevertheless, the key for the reconstruction process lies within the query and filtering criteria of the Search function which we build following the RIC construction RULES.

In [63]:
class FuturesRICs:
    MONTH_CODES = {'F': '01', 'G': '02', 'H': '03', 'J': '04', 'K': '05', 'M': '06',
                   'N': '07', 'Q': '08', 'U': '09', 'V': '10', 'X': '11', 'Z': '12'}
    ASSET_CATEGORY = "Future"
    DATE_FORMAT = "%Y-%m-%d"
 
    def get_futures(self, underlying, month, year):
        month_num, month_code = self._get_month_number_and_code(month)
        month_last_day = calendar.monthrange(year, int(month_num))[1]-1
        # we define two possible year_codes to account for RIC structure change
        year_codes = [str(year)[-1], str(year)[-2:]]
        for year_code in year_codes:
            query = f'{underlying}{month_code}{year_code}*'
            filter_criteria = self._build_filter_criteria(year, month_num, month_last_day, query)
            response = self._search_futures(query, filter_criteria)
            if response is not None and not response.empty:
                return response
        print(f'No futures contract for {underlying} expiring on {month_num} month of {year}')
        return None
 
    def _search_futures(self, query, filter_criteria):
        response = rd.discovery.search(
            view=rd.discovery.Views.DERIVATIVE_QUOTES,
            query=query,
            select="DocumentTitle, RIC, ExchangeCode, ExpiryDate, " +
                   "UnderlyingQuoteRIC, RCSAssetCategoryLeaf, RetireDate",
            filter=filter_criteria
        )
        return response if not response.empty else None
 
    def _get_month_number_and_code(self, month):
        if isinstance(month, int) or month.isnumeric():
            month_num = f'{int(month):02}'
            month_code = next((code for code, num in self.MONTH_CODES.items() if num == month_num), None)
            if month_code is None:
                raise ValueError(f"Invalid numeric month: {month}")
        else:
            month_code = month.upper()
            month_num = self.MONTH_CODES.get(month_code)
            if month_num is None:
                raise ValueError(f"Invalid month code: {month_code}")
        return month_num, month_code
 
    def _build_filter_criteria(self, year, month_num, month_last_day, query):
        return (
            f"RCSAssetCategoryLeaf eq '{self.ASSET_CATEGORY}' and "
            f"ExpiryDate ge {year-1}-{month_num}-{month_last_day} and "
            f"ExpiryDate le {year+1}-{month_num}-{month_last_day} and "
            f"(RIC xeq '{query[:-1]}' or RIC xeq '{query[:-1]}^{str(year)[-2]}')"
        )

Below we initialise the function.

In [64]:
fr = FuturesRICs()

Finally, let's request for example an LCO future expiring on December 2015 using our object.

In [68]:
lco_expired = fr.get_futures('LCO', 12, 2015)
lco_expired

,DocumentTitle,RIC,ExchangeCode,ExpiryDate,RCSAssetCategoryLeaf,RetireDate
0,ICE Brent Crude Electronic Energy Future Dec 2...,LCOZ5^1,IEU,2015-11-13,Commodity Future,2015-11-17


As we have the RIC, now we request the historical prices going back to 2010.

In [46]:
lco_prices = rd.get_history(universe = lco_expired['RIC'][0], fields = ['SETTLE','TRDPRC_1', 'BID', 'ASK'],  start = '2010-01-01', count = 10000)
lco_prices

LCOG5^1,SETTLE,TRDPRC_1,BID,ASK
Date,,,,
2010-01-04,94.11,<NA>,<NA>,<NA>
2010-01-05,94.59,<NA>,<NA>,<NA>
2010-01-06,95.99,<NA>,<NA>,<NA>
2010-01-07,95.46,<NA>,<NA>,<NA>
2010-01-08,95.84,<NA>,<NA>,<NA>
...,...,...,...,...
2015-01-09,50.11,49.95,49.92,49.95
2015-01-12,47.43,47.2,47.16,47.25
2015-01-13,46.59,46.79,46.63,46.79


Below we request also for different underlings with valid expiration periods (It should be noted that different futures contracts have differing contract specifications, including the contract month. So that specifications should be considered when requesting an option contract RICs. To review futures contract specifications for an underlying we can search the continuation RIC in Workspace and click on the Contract Specification Tab).

In [47]:
print(fr.get_futures('AD', 3, 2000)['RIC'][0])
print(fr.get_futures('ES', 3, 2002)['RIC'][0])
print(fr.get_futures('FLG', 9, 2004)['RIC'][0])
print(fr.get_futures('JY', 9, 2006)['RIC'][0])
print(fr.get_futures('NG', 5, 2008)['RIC'][0])
print(fr.get_futures('S', 1, 2010)['RIC'][0])
print(fr.get_futures('W', 7, 2012)['RIC'][0])
print(fr.get_futures('VX', 8, 2014)['RIC'][0])
print(fr.get_futures('SB', 10, 2016)['RIC'][0])
print(fr.get_futures('JGB', 3, 2018)['RIC'][0])
print(fr.get_futures('SON3', 6, 2020)['RIC'][0])
print(fr.get_futures('FFI', 9, 2022)['RIC'][0])
print(fr.get_futures('FDX', 12, 2023)['RIC'][0])
print(fr.get_futures('ES', 3, 2023)['RIC'][0])
print(fr.get_futures('LCO', 2, 2026)['RIC'][0])

ADH0^0
ESH2^0
FLGU4^0
JYU6^0
NGK8^0
SF0^1
WN2^1
VXQ4^1
SBV6^1
JGBH8^1
SON3M0^2
FFIU2^2
FDXZ3^2
ESH3^2
LCOG6


In [11]:
rd.close_session()